In [5]:
!python resize_images.py

resize on 8 CPUs
[100/74381] Resized the images and saved.
[200/74381] Resized the images and saved.
[300/74381] Resized the images and saved.
[400/74381] Resized the images and saved.
[500/74381] Resized the images and saved.
[600/74381] Resized the images and saved.
[700/74381] Resized the images and saved.
[800/74381] Resized the images and saved.
[900/74381] Resized the images and saved.
[1000/74381] Resized the images and saved.
[1100/74381] Resized the images and saved.
[1200/74381] Resized the images and saved.
[1300/74381] Resized the images and saved.
[1400/74381] Resized the images and saved.
[1500/74381] Resized the images and saved.
[1600/74381] Resized the images and saved.
[1700/74381] Resized the images and saved.
[1800/74381] Resized the images and saved.
[1900/74381] Resized the images and saved.
[2000/74381] Resized the images and saved.
[2100/74381] Resized the images and saved.
[2200/74381] Resized the images and saved.
[2300/74381] Resized the images and saved.
[25

In [1]:
import torch

import os

'''
If you are using Colab, make sure to add the relevant CIR folder to the system path. Alternatively, 
 you can adjust the import statements below based on the location of your files.
'''

from data_loader import get_loader
from utils import Ranker
from model_arch import Model, Criterion, train, val

In [2]:
# You can change hyper-parameters to achieve higher retrieval performance
class Args():
    def __init__(self):
        # Dataset
        self.data_root = "./" # Remember to change the path to your data
        self.data_set = "dress"
        self.image_root = os.path.join(self.data_root, 'data/resized_images/')
        self.caption_path = os.path.join(self.data_root, 'data/captions/cap.{}.{}.json')
        self.split_path = os.path.join(self.data_root, 'data/image_splits/split.{}.{}.json')

        # Model
        self.embed_dim = 512
        self.vision_feature_dim = 512
        self.text_feature_dim = 512

        # Training
        self.log_step = 15
        self.batch_size = 64
        self.learning_rate = 0.001
        self.num_workers = 4
        self.epochs = 3
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args = Args()

In [3]:
# Build data loader
data_loader = get_loader(
    args.image_root.format(args.data_set),
    args.caption_path.format(args.data_set, 'train'),
    args.batch_size,
    shuffle=True, 
    return_target=True, 
    num_workers=args.num_workers,
)
data_loader_dev = get_loader(
    args.image_root.format(args.data_set),
    args.caption_path.format(args.data_set, 'val'),
    args.batch_size, 
    shuffle=False, 
    return_target=True, 
    num_workers=args.num_workers,
)

In [18]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/d4/55/90db48d85f7689ec6f81c0db0622d704306c5284850383c090e6c7195a5c/pip-24.2-py3-none-any.whl.metadata
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install git+https://github.com/openai/CLIP.git --user

  Cloning https://github.com/openai/CLIP.git to /private/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/pip-req-build-9gwva5m5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/pip-req-build-9gwva5m5
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=f67fad72b11695e3ed46279ca40f755e4dd9176de621727c625ee48a518c889c
  Stored in directory: /private/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/pip-ephem-wheel-cache-ip7wvibg/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstall

In [4]:
# Build model, criterion, oprimizer, evaluator
model = Model(args.vision_feature_dim, args.text_feature_dim, args.embed_dim)
model.to(args.device)
model.train()
criterion = Criterion()
current_lr = args.learning_rate
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=current_lr)
ranker = Ranker(root=args.image_root.format(args.data_set), image_split_file=args.split_path.format(args.data_set, 'val'), transform=None, num_workers=args.num_workers)

In [5]:
# Start training
best_score = 0
for epoch in range(args.epochs):
    train(data_loader, model, criterion, optimizer, args.log_step)
    best_score = val(data_loader_dev, model, ranker, best_score)
print(best_score)

training loss: 4.227
training loss: 3.080
training loss: 2.788
training loss: 2.769
training loss: 2.704
training loss: 2.543
training loss: 2.522
updating emb
emb updated
-----------------------------------------------------------------------------
| score  0.86237 /  0.86237 
-----------------------------------------------------------------------------
best_dev_score: 0.8623731261871234
training loss: 2.196
training loss: 2.120
training loss: 2.221
training loss: 2.307
training loss: 2.305
training loss: 2.241
training loss: 2.110
updating emb
emb updated
-----------------------------------------------------------------------------
| score  0.87603 /  0.87603 
-----------------------------------------------------------------------------
best_dev_score: 0.8760284751630247
training loss: 2.006
training loss: 2.163
training loss: 1.784
training loss: 1.987
training loss: 2.069
training loss: 2.078
training loss: 1.879
updating emb
emb updated
--------------------------------------------

In [12]:
# Assuming 'model' is your trained PyTorch model
torch.save(model.state_dict(),'trained_model.pth')
